# One Max Problem

##### One Max Problem — DEAP 1.3.1 documentation. (2020). Retrieved 11 August 2020, from https://deap.readthedocs.io/en/master/examples/ga_onemax.html

Utilizaremos la librería de Python DEAP que es un framework de cómputo evolutivo para hacer prototipos rápidamente y probar ideas. Busca hacer algoritmos explícitos y estructuras de datos transparentes. Trabaja en perfecta armonía con mecanismos de paralelismo como $\texttt{multiprocessing}$ y $\texttt{SCOOP}$.

## Ejemplo que maximiza la suma de una lista de enteros.

Por ejemplo se comienza con un conjunto de listas de 0s y 1s aleatorios (e.g. [0,1,0,0,0]), y se busca evolucionar hasta que algún elemento llegue a ser de la forma [1,1,1,1,1].

Hay muchos parámetros que se pueden ajustar como la logitud de las listas, el tamaño inicial de la población, etc.

El problema a resolver ayuda a entender algunas de las posibilidades de este framework e ilustra de manera concisa el concepto de máquinas evolutivas en general. Primero importaremos los módulos necesarios



In [ ]:
import random

from deap import base
from deap import creator
from deap import tools

## Creator 

Debido a que la estructura exacta de los individuos en los algoritmos genéticos depende fuertemente en la problema que se quiere resolver, DEAP no tiene una estructura explícita. Lo que si hace es darnos una manera conveniente para crear "contenedores" de atributos, asociarlos con su aptitud (fitness) llamado $\texttt{deap.creator}$. 

$\texttt{creator}$ es una fábrica de clases que las va generando durante el tiempo de ejecución. El primer argumento es el nombre de la clase, seguido de la clase base de la cual heredará, y finalmente cualquier argumento que queramos pasar a la hora de instanciar la misma.

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

## Toolbox

Utilizaremos la clase a la medida que acamos de crear y haremos uso de Toolbox para crear a nuestro individuos así como a toda la población.

Herramientas para crear a los individuos ($\texttt{random.randint}$), los individuos mismos y la población, quedará registarada bajo un contenedor llamado $\texttt{Toolbox}$. Tiene dos métodos para agregar o remover contenido, register() y unregister().

Es importante mencionar que de forma análoga a $\texttt{creator.create}$, el primer argumento es el nombre de nuestra función, luego la función que encapsula y finalmente los argumentos de la función. Al registrar las herramientas en el toolbox, solamente crea alias a la funciones ya existentes y congela parte de sus argumentos. Con esto podemos dejar algunos de los argumentos abiertos para modificarlos después. Por ejemplo, en el caso de la población.

In [3]:
toolbox = base.Toolbox()

# Attribute generator 
#                      define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
toolbox.register("attr_bool", random.randint, 0, 1)

# Structure initializers
#                         define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 50)

# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# the goal ('fitness') function to be maximized
def evalOneMax(individual):
    return sum(individual),

#----------
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", evalOneMax)

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.0555)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selRoulette,fit_attr='fitness')

#----------


In [1]:

def main():
    random.seed(64)

    # create an initial population of 500 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=5000)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.9, 0.005
    
    print("Start of evolution")
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while max(fits) < 50 and g < 1000:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        print("  Evaluated %i individuals" % len(invalid_ind))
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
    
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

if __name__ == "__main__":
    main()

Start of evolution
  Evaluated 5000 individuals
-- Generation 1 --
  Evaluated 4496 individuals
  Min 14.0
  Max 37.0
  Avg 25.65
  Std 3.53274680666476
-- Generation 2 --
  Evaluated 4519 individuals
  Min 15.0
  Max 38.0
  Avg 26.103
  Std 3.485970596548392
-- Generation 3 --
  Evaluated 4522 individuals
  Min 15.0
  Max 39.0
  Avg 26.5842
  Std 3.4771986368339767
-- Generation 4 --
  Evaluated 4456 individuals
  Min 14.0
  Max 40.0
  Avg 27.0782
  Std 3.455153362732259
-- Generation 5 --
  Evaluated 4444 individuals
  Min 13.0
  Max 40.0
  Avg 27.4962
  Std 3.4987977306497533
-- Generation 6 --
  Evaluated 4455 individuals
  Min 15.0
  Max 39.0
  Avg 27.9278
  Std 3.4701854647842545
-- Generation 7 --
  Evaluated 4550 individuals
  Min 16.0
  Max 39.0
  Avg 28.3584
  Std 3.4438277308831866
-- Generation 8 --
  Evaluated 4505 individuals
  Min 17.0
  Max 40.0
  Avg 28.7908
  Std 3.399681655684823
-- Generation 9 --
  Evaluated 4487 individuals
  Min 17.0
  Max 42.0
  Avg 29.1844
  St